Generate the FI curve of SST neurons in the presence or absence of oxytocin. Based on https://nest-simulator.readthedocs.io/en/nest-2.20.1/auto_examples/if_curve.html .

In [1]:
import numpy
import nest

Here we define which model and the neuron parameters to use for measuring
the transfer function.



In [2]:
model = 'iaf_psc_delta'
paramsBL = {'V_m': -70.0,
           'E_L': -60.802, # Maldonado et al 2020
           'C_m': 200.0,
           'tau_m': 106.6, # Maldonado et al 2020
           't_ref': 2.0,
           'V_th': -32.0, # Maldonado et al 2020
           'V_reset': -60.0}
paramsOT = {'V_m': -70.0,
           'E_L': -56.3, # Maldonado et al 2020
           'C_m': 200.0,
           'tau_m': 106.6, # Maldonado et al 2020
           't_ref': 2.0,
           'V_th': -33.0, # Maldonado et al 2020
           'V_reset': -60.0}

class IF_curve():

    t_inter_trial = 200.  # Interval between two successive measurement trials
    t_sim = 4000.         # Duration of a measurement trial
    n_neurons = 100       # Number of neurons
    n_threads = 4         # Nubmer of threads to run the simulation

    def __init__(self, model, params=False):
        self.model = model
        self.params = params
        self.build()
        self.connect()

    def build(self):
        #######################################################################
        #  We reset NEST to delete information from previous simulations
        # and adjust the number of threads.

        nest.ResetKernel()
        nest.SetKernelStatus({'local_num_threads': self.n_threads})

        #######################################################################
        # We set the default parameters of the neuron model to those
        # defined above and create neurons and devices.

        if self.params:
            nest.SetDefaults(self.model, self.params)
        self.neuron = nest.Create(self.model, self.n_neurons)
        self.noise = nest.Create('noise_generator')
        self.spike_detector = nest.Create('spike_detector')

    def connect(self):
        #######################################################################
        # We connect the noisy current to the neurons and the neurons to
        # the spike detectors.

        nest.Connect(self.noise, self.neuron, 'all_to_all')
        nest.Connect(self.neuron, self.spike_detector, 'all_to_all')

    def output_rate(self, mean, std):
        self.build()
        self.connect()

        #######################################################################
        # We adjust the parameters of the noise according to the current
        # values.

        #nest.SetStatus(self.noise, [{'mean': mean, 'std': std, 'start': 0.0,
        #                             'stop': 1000., 'origin': 0.}])
        nest.SetStatus(self.neuron, "I_e", 0.0 + mean)
        # We simulate the network and calculate the rate.

        nest.Simulate(self.t_sim)
        rate = nest.GetStatus(self.spike_detector, 'n_events')[0] * 1000.0 \
            / (1. * self.n_neurons * self.t_sim)
        return rate

    def compute_transfer(self, i_mean=(400.0, 900.0, 50.0),
                         i_std=(0.0, 600.0, 50.0)):
        #######################################################################
        # We loop through all possible combinations of `(I_mean, I_sigma)`
        # and measure the output rate of the neuron.

        self.i_range = numpy.arange(*i_mean)
        self.std_range = numpy.arange(*i_std)
        self.rate = numpy.zeros((self.i_range.size, self.std_range.size))
        nest.set_verbosity('M_WARNING')
        for n, i in enumerate(self.i_range):
            # print('I  =  {0}'.format(i))
            for m, std in enumerate(self.std_range):
                self.rate[n, m] = self.output_rate(i, std)



In [3]:
transferBL = IF_curve(model, paramsBL)
transferBL.compute_transfer(i_mean=(0.0, 140.0, 5.0))
transferOT = IF_curve(model, paramsOT)
transferOT.compute_transfer(i_mean=(0.0, 140.0, 5.0))

After the simulation is finished we store the data into a file for
later analysis.



In [4]:
numpy.save('SIM_OUT/BLrate.npy' , transferBL.rate)
numpy.save('SIM_OUT/BLrange.npy' , transferBL.i_range)
numpy.save('SIM_OUT/BLstd_range.npy' , transferBL.std_range)
numpy.save('SIM_OUT/OTrate.npy' , transferOT.rate)
numpy.save('SIM_OUT/OTrange.npy' , transferOT.i_range)
numpy.save('SIM_OUT/OTstd_range.npy' , transferOT.std_range)